In [1]:
import requests
import time
import numpy as np
import pickle
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# URL-адреса новостных сайтов
news_sites = [
    "https://ru.investing.com/news/latest-news",
    "https://bcs-express.ru/category/vazhnoe",
    "https://ru.tradingview.com/news/markets/",
    "https://quote.ru/category/News"
]

# Ключевые слова для фильтрации новостей
keywords = ["технологии", "наука", "политика"]

In [ ]:
# Функция для парсинга новостей
def parse_news(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    print(soup)
    news_items = []

    # Пример извлечения новостей (зависит от структуры сайта)
    for item in soup.find_all('article'):
        title = item.find('h2').text
        content = item.find('p').text
        date = item.find('time').text
        news_items.append({'title': title, 'content': content, 'date': date})

    return news_items


parse_news(news_sites[0])

In [4]:
# Функция для фильтрации новостей по ключевым словам
def filter_news(news_items, keywords):
    filtered_news = []
    for item in news_items:
        if any(keyword in item['content'] for keyword in keywords):
            filtered_news.append(item)
    return filtered_news

In [5]:
# Функция для сохранения новостей в векторную БД
def save_news_to_vector_db(news_items):
    # Пример использования TF-IDF для преобразования текста в векторы
    vectorizer = TfidfVectorizer()
    texts = [item['content'] for item in news_items]
    vectors = vectorizer.fit_transform(texts).toarray()

    # Сохранение векторов в файл (вместо векторной БД для примера)
    with open('news_vectors.pkl', 'wb') as f:
        pickle.dump(vectors, f)

In [6]:
# Основная функция для выполнения пайплайна
def run_pipeline():
    all_news = []
    for site in news_sites:
        news_items = parse_news(site)
        all_news.extend(news_items)

    filtered_news = filter_news(all_news, keywords)
    save_news_to_vector_db(filtered_news)

    # Удаление старых новостей (старше 30 дней)
    current_date = datetime.now()
    thirty_days_ago = current_date - timedelta(days=30)
    filtered_news = [item for item in filtered_news if datetime.strptime(item['date'], '%Y-%m-%d') > thirty_days_ago]

    print(f"Обновлено {len(filtered_news)} новостей.")

# Планирование задачи на выполнение каждый день в 23:59
schedule.every().day.at("23:59").do(run_pipeline)

# Бесконечный цикл для выполнения запланированных задач
while True:
    schedule.run_pending()
    time.sleep(1)


NameError: name 'schedule' is not defined